<a href="https://colab.research.google.com/github/MaricelaMH/SIMULACION-II/blob/main/Linea_de_espera_combi_proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SIMULACIÓN DE UNA PARADA DE COMBIS

Contexto del problema: En Naucalpan de Juarez, Estado de México, hay una parada de combis en el paradero de Cuatro Caminos donde los pasajeros se forman para esperar la próxima llegada de una combi. Esta parada sigue una disciplina de servicio primero en entrar, primero en salir (FIFO), donde la combi recoge a los pasajeros en el orden en que llegaron. Para ello realizaremos una simulación para lograr entender mejor el comportamiento de nuestro sistema y asi tomar desiciones sobre los horarios y capacidad de las combis para evitar tiempos de esperas excesivos parea los pasajeros.

1. Para simular lo anterior, usaremos la libreria simpy por lo que la importaremos junto con las demas librerias que usaremos.

In [1]:
!pip install simpy

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from random import random, seed
import pandas as pd
import simpy

2. Definimos nuestros parámetros

In [21]:
# Parámetros del sistema de combis
no_combis = 1            # Cantidad de combis disponibles en la parada
tc_min = 1                  # Tiempo mínimo de abordaje por pasajero (minutos)
tc_max = 2                  # Tiempo máximo de abordaje por pasajero (minutos)
ll = 2 / 1                  # Tasa de llegada de los pasajeros (2 cada minuto)
mu = 1 / 15                 # Tasa de llegada de combis (1 cada 15 minutos)
clientes = 100               # Total de pasajeros a simular
cifras = 2                  # Cantidad de cifras decimales para mostrar
cap_combi = 19              # Capacidad máxima de pasajeros de cada combi

In [22]:
TLL, TA, TS = [], [], []   # Listas para tiempos de llegada de los pasajeros, abordaje, y tiempo de salida de la combi

In [23]:
# Parámetro de simulación
semilla = 13
seed(semilla)

3. Creamos una función que nos ayudara a simular la llegada de los pasajeros a la parada de combis las cuales suelen seguir una distribución exponencial, por lo que la media estara definida como:

$$ \mu = \frac{1}{ll}  $$

donde $ll$ es el tiempo promedio de llegada de los pasajeros

In [18]:
# Función para simular cuando los pasajeros llegan a la parada de combis.
def parada_combis(env, parada, num_pasajeros, texto=True):
    for i in range(num_pasajeros):
        # Tiempo hasta la próxima llegada del pasajero siguiendo distribución exponencial
        mu = 1 / ll
        tll = np.random.exponential(mu)
        yield env.timeout(tll)
        TLL.append(env.now)  # Almacenamos el tiempo de llegada del pasajero
        if texto:
            print(f'----Ha llegado el pasajero número {i} en el minuto {round(env.now, cifras)}----')
        env.process(abordaje(env, parada, i, texto))

4. Creamos nuevamente otra función que nos ayudara a simular el

In [ ]:
def llegada_autobus(env, parada, texto=False):
    global mu
    while True:
        # Tiempo entre llegadas de autobuses siguiendo distribución exponencial
        t_llegada_autobus = np.random.exponential(scale=1/mu)
        yield env.timeout(t_llegada_autobus)

        num_pasajeros = len(parada.queue)  # Pasajeros esperando en la cola
        if texto:
            print(f'Autobús llega a la parada a los {round(env.now, cifras)} minutos')

        # Determinar cuántos pasajeros recogerá el autobús
        if num_pasajeros > 0:
            pasajeros_a_recoger = min(num_pasajeros, capacidad_autobus)
            for _ in range(pasajeros_a_recoger):
                parada.release(parada.request())  # Se simula la subida de los pasajeros al autobús
            if texto:
                print(f'Autobús recoge a {pasajeros_a_recoger} pasajeros en la parada a los {round(env.now, cifras)} minutos')


5. Creamos una función llamada abordaje la cual ayudara a simular el tiempo que le lleva al pasajero abordar una combi , donde utilizaremos una distribución uniforme para determinar el tiempo mínimo y máximo de abordaje del pasajero.

In [20]:
def abordaje(env, parada, num_pasajero, texto=True):
    global tc_min, tc_max, cifras, dur_abordaje, LC, LS
    # Solicitamos abordar una combi
    with parada.request() as request:  # Solicitamos una combi
        yield request  # Hace que el pasajero espere hasta que pueda abordar la combi
        # Tiempo de abordaje
        tc = np.random.uniform(tc_min, tc_max)  # Tiempo de corte aleatorio
        dur_abordaje += tc
        LC.append(tc)
        if texto:
            print(f'****Pasajero {num_pasajero} inicia el abordaje en el minuto {round(env.now, cifras)}****')
        yield env.timeout(tc)
        LS.append(env.now)
        if texto:
            print(f'xxxPasajero {num_pasajero} termina el abordaje en el minuto {round(env.now, cifras)}xxx')
